## **Model Tuning Notebook**

In [1]:
from numba import cuda
import tensorflow as tf

def clear_gpu_memory():
    device = cuda.get_current_device()
    device.reset() # dump the memory contents to free up the memory (it accumulates over the session)
    
# CUDA (Nvidia GPU Computing)
if len(tf.config.list_physical_devices('GPU')) > 0:
    gpus = tf.config.list_physical_devices('GPU')
    print("Num GPUs Available: ", len(gpus))
    
    clear_gpu_memory()
    
    tf.config.experimental.set_memory_growth(gpus[0], True)

    tf.config.set_logical_device_configuration(
    gpus[0], 
    [tf.config.LogicalDeviceConfiguration(memory_limit=8192)])  # limit to 4GB

    tf.compat.v1.disable_eager_execution()


Num GPUs Available:  1


In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('../')

from src.models.models import *
from src.utils.modeling import *
from src.utils.preproc import *

from keras.models import Model
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam

from sklearn.metrics import classification_report

In [3]:
# because the utils in the src are designed to be run from the root of the project,
# and by default jupyter runs from the notebook directory we need to change the working directory to the root

def find_project_root(filename=".git"): # .git is located in the root of the project
    current_dir = os.getcwd()
    while current_dir != os.path.dirname(current_dir): # stops only when at the root (moves up 1 level each iteration)
        if filename in os.listdir(current_dir):
            return current_dir
        current_dir = os.path.dirname(current_dir)

project_root = find_project_root()
os.chdir(project_root)  # change the working directory to the project root

print("Project root:", project_root, "CWD:", os.getcwd())

Project root: d:\deep_learning_project CWD: d:\deep_learning_project


### **Binary Classification Models Fine-Tuning**

#### **Traditional**

#### **Pre-Trained**

**VGG 16**

In [4]:
def binary_classification_vgg16_model(input_shape):
    
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # train some layers and freeze others
    for layer in base_model.layers[:-4]:
        layer.trainable = False
    for layer in base_model.layers[-4:]:
        layer.trainable = True

    model = Sequential()
    
    model.add(base_model)
    
    model.add(Flatten())
    
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(1, activation='sigmoid'))
    
    # low learning rate for fine tuning
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
if len(tf.config.list_physical_devices('GPU')) > 0:
    clear_gpu_memory()

train_gen, val_gen, test_gen, class_weights = preproc_pipeline(desired_magnification='200X', 
                                                    image_resolution=(224, 224), 
                                                    classification_type='binary')

vgg16 = binary_classification_vgg16_model((224, 224, 3))

fitted_vgg16 = train_model(train_gen, val_gen, vgg16, class_weights=class_weights, epochs=10)

test_loss, test_acc = fitted_vgg16.evaluate(X_test, y_test)
print(f'Test loss: ', test_loss)
print(f'Test accuracy: ', test_acc)

Epoch 1/10
44/44 [==============================] - ETA: 0s - batch: 21.5000 - size: 32.0000 - loss: 0.6080 - accuracy: 0.6619

c:\Users\vic\miniconda3\envs\tf\lib\site-packages\keras\engine\training_v1.py:2332: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


44/44 [==============================] - 58s 569ms/step - batch: 21.5000 - size: 32.0000 - loss: 0.6080 - accuracy: 0.6619 - val_loss: 0.5586 - val_accuracy: 0.6755
Epoch 2/10
44/44 [==============================] - 17s 366ms/step - batch: 21.5000 - size: 32.0000 - loss: 0.4588 - accuracy: 0.7663 - val_loss: 0.3790 - val_accuracy: 0.8146
Epoch 3/10
44/44 [==============================] - 16s 356ms/step - batch: 21.5000 - size: 32.0000 - loss: 0.4083 - accuracy: 0.8132 - val_loss: 0.5486 - val_accuracy: 0.7053
Epoch 4/10
44/44 [==============================] - 17s 370ms/step - batch: 21.5000 - size: 32.0000 - loss: 0.2893 - accuracy: 0.8736 - val_loss: 0.2648 - val_accuracy: 0.8775
Epoch 5/10
44/44 [==============================] - 16s 350ms/step - batch: 21.5000 - size: 32.0000 - loss: 0.3044 - accuracy: 0.8636 - val_loss: 0.2750 - val_accuracy: 0.8642
Epoch 6/10
44/44 [==============================] - 16s 358ms/step - batch: 21.5000 - size: 32.0000 - loss: 0.2395 - accuracy: 0.89

<i> Data augmentation clearly improves the model's performance. </i>

In [4]:
def multiclass_classification_vgg16_model(input_shape):
    
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # train some layers and freeze others
    for layer in base_model.layers[:-7]:
        layer.trainable = False
    for layer in base_model.layers[-7:]:
        layer.trainable = True

    model = Sequential()
    
    model.add(base_model)
    
    model.add(Flatten())
    
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(8, activation='softmax'))
    
    # low learning rate for fine tuning
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
train_gen, val_gen, test_gen, class_weights = preproc_pipeline(desired_magnification='200X', 
                                                    image_resolution=(224, 224), 
                                                    classification_type='multiclass',
                                                    use_data_augmentation=True,
                                                    augmented_images_per_image=6)

vgg16 = multiclass_classification_vgg16_model((224, 224, 3))

fitted_vgg16 = train_model(train_gen, val_gen, vgg16, class_weights=class_weights, epochs=30, early_stopping_patience=5)

get_classification_report(fitted_vgg16, X_test, y_test)

Number of training images before data augmentation: 1408
Number of training images after data augmentation: 9856
Epoch 1/30
308/308 [==============================] - ETA: 0s - batch: 153.5000 - size: 32.0000 - loss: 1.7375 - accuracy: 0.2993

c:\Users\vic\miniconda3\envs\tf\lib\site-packages\keras\engine\training_v1.py:2332: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


308/308 [==============================] - 199s 492ms/step - batch: 153.5000 - size: 32.0000 - loss: 1.7375 - accuracy: 0.2993 - val_loss: 1.6420 - val_accuracy: 0.3742
Epoch 2/30
308/308 [==============================] - 144s 466ms/step - batch: 153.5000 - size: 32.0000 - loss: 1.0866 - accuracy: 0.5148 - val_loss: 1.2366 - val_accuracy: 0.5232
Epoch 3/30
308/308 [==============================] - 147s 472ms/step - batch: 153.5000 - size: 32.0000 - loss: 0.7633 - accuracy: 0.6423 - val_loss: 0.9014 - val_accuracy: 0.6490
Epoch 4/30
308/308 [==============================] - 152s 491ms/step - batch: 153.5000 - size: 32.0000 - loss: 0.6147 - accuracy: 0.7111 - val_loss: 0.6449 - val_accuracy: 0.7152
Epoch 5/30
308/308 [==============================] - 176s 569ms/step - batch: 153.5000 - size: 32.0000 - loss: 0.4997 - accuracy: 0.7468 - val_loss: 0.7463 - val_accuracy: 0.7417
Epoch 6/30
308/308 [==============================] - 189s 612ms/step - batch: 153.5000 - size: 32.0000 - loss:

c:\Users\vic\miniconda3\envs\tf\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


              precision    recall  f1-score   support

           0       0.94      0.94      0.94        16
           1       0.96      0.85      0.90       135
           2       0.87      0.85      0.86        39
           3       0.65      0.88      0.75        25
           4       0.81      0.83      0.82        30
           5       0.79      0.95      0.86        20
           6       0.69      0.69      0.69        16
           7       0.91      1.00      0.95        21

    accuracy                           0.86       302
   macro avg       0.83      0.87      0.85       302
weighted avg       0.88      0.86      0.87       302



In [4]:
def multiclass_classification_vgg16_model(input_shape):
    
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # train some layers and freeze others
    for layer in base_model.layers[:-7]:
        layer.trainable = False
    for layer in base_model.layers[-7:]:
        layer.trainable = True

    model = Sequential()
    
    model.add(base_model)
    
    model.add(Flatten())
    
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(8, activation='softmax'))
    
    # low learning rate for fine tuning
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [5]:
train_gen, val_gen, test_gen, class_weights = preproc_pipeline(desired_magnification='200X', 
                                                    image_resolution=(224, 224), 
                                                    classification_type='multiclass',
                                                    use_data_augmentation=True,
                                                    augmented_images_per_image=6)

vgg16 = multiclass_classification_vgg16_model((224, 224, 3))

fitted_vgg16 = train_model(train_gen, val_gen, vgg16, class_weights=class_weights, epochs=30, early_stopping_patience=5)

get_classification_report(fitted_vgg16, X_test, y_test)

Number of training images before data augmentation: 1408
Number of training images after data augmentation: 9856
Epoch 1/30
308/308 [==============================] - ETA: 0s - batch: 153.5000 - size: 32.0000 - loss: 2.0810 - accuracy: 0.1444

c:\Users\vic\miniconda3\envs\tf\lib\site-packages\keras\engine\training_v1.py:2332: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


308/308 [==============================] - 177s 461ms/step - batch: 153.5000 - size: 32.0000 - loss: 2.0810 - accuracy: 0.1444 - val_loss: 1.9579 - val_accuracy: 0.2219
Epoch 2/30
308/308 [==============================] - 143s 463ms/step - batch: 153.5000 - size: 32.0000 - loss: 1.9442 - accuracy: 0.1763 - val_loss: 1.5821 - val_accuracy: 0.4205
Epoch 3/30
308/308 [==============================] - 128s 413ms/step - batch: 153.5000 - size: 32.0000 - loss: 1.4297 - accuracy: 0.3977 - val_loss: 1.3358 - val_accuracy: 0.4437
Epoch 4/30
308/308 [==============================] - 129s 417ms/step - batch: 153.5000 - size: 32.0000 - loss: 1.0566 - accuracy: 0.5172 - val_loss: 1.1213 - val_accuracy: 0.5099
Epoch 5/30
308/308 [==============================] - 131s 423ms/step - batch: 153.5000 - size: 32.0000 - loss: 0.7952 - accuracy: 0.6024 - val_loss: 1.1534 - val_accuracy: 0.5662
Epoch 6/30
308/308 [==============================] - 130s 420ms/step - batch: 153.5000 - size: 32.0000 - loss:

c:\Users\vic\miniconda3\envs\tf\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


              precision    recall  f1-score   support

           0       1.00      0.94      0.97        16
           1       0.91      0.79      0.84       135
           2       0.91      0.77      0.83        39
           3       0.57      0.68      0.62        25
           4       0.83      0.80      0.81        30
           5       0.54      0.95      0.69        20
           6       0.71      0.94      0.81        16
           7       0.86      0.90      0.88        21

    accuracy                           0.81       302
   macro avg       0.79      0.85      0.81       302
weighted avg       0.84      0.81      0.82       302



In [6]:
train_gen, val_gen, test_gen, class_weights = preproc_pipeline(desired_magnification='200X', 
                                                    image_resolution=(224, 224), 
                                                    classification_type='binary',
                                                    use_data_augmentation=False,
                                                    augmented_images_per_image=6)

In [10]:
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight

def resize_and_append(image_path, label, X, y, img_size):
    """
    Resize an image and append it along with its label to the provided lists.
    
    Parameters:
        - image_path (str): The file path to the image to be resized.
        - label (any): The label associated with the image.
        - X (list): The list to which the resized image array will be appended.
        - y (list): The list to which the label will be appended.
        - img_size (tuple): The target size for resizing the image (width, height).
        
    Returns:
        None
    """
    with Image.open(image_path) as img:
        img_resized = img.resize(img_size)
        img_array = np.array(img_resized)
        
        X.append(img_array)
        y.append(label)
    
def label_encode(y_train, y_test, y_val):
    """
    Encodes the labels of the training, testing, and validation datasets using label encoding.
    
    Parameters:
        - y_train (array-like): The labels for the training dataset.
        - y_test (array-like): The labels for the testing dataset.
        - y_val (array-like): The labels for the validation dataset.
        
    Returns:
        - y_train (numpy.ndarray): Encoded training set labels.
        - y_test (numpy.ndarray): Encoded testing set labels.
        - y_val (numpy.ndarray): Encoded validation set labels.
    """
    le = LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_test = le.fit_transform(y_test)
    y_val = le.fit_transform(y_val)
        
    return y_train, y_test, y_val

def load_and_preprocess_data(csv_path, desired_magnification, image_resolution, label_column):
    """
    Load and preprocess image data from the CSV file containing the image metadata.
    This function reads image data paths and labels from the CSV file, filters the data based on the desired magnification,
    resizes the images to the specified resolution, sorts the data into training, testing, and validation arrays.
    The images are then normalized, and the labels are encoded.
    
    Parameters:
        - csv_path (str): Path to the image metadata CSV.
        - desired_magnification (int): The magnification level to filter the images (40X, 100X, 200X, 400X).
        - image_resolution (tuple): The desired resolution to resize the images (width, height).
        - label_column (str): The name of the column in the CSV file that contains the labels ('Benign or Malignant' or 'Cancer Type').
        
    Returns:
        - X_train (numpy.ndarray): Training set images.
        - y_train (numpy.ndarray): Training set labels.
        - X_test (numpy.ndarray): Testing set images.
        - y_test (numpy.ndarray): Testing set labels.
        - X_val (numpy.ndarray): Validation set images.
        - y_val (numpy.ndarray): Validation set labels.
    """
    df = pd.read_csv(csv_path)
    
    # select only the rows for the selected magnification (40X, 100X, 200X, 400X)
    df_filtered = df[df['Magnification'] == desired_magnification]
    
    X_train, y_train = [], []
    X_test, y_test = [], []
    X_val, y_val = [], []
    
    # it is necessary to use the updated_image_data.csv file to get the correct path to the images
    for boda, row in df_filtered.iterrows():
        image_path = row['path_to_image']
        label = row[label_column]
        if 'train' in image_path:
            resize_and_append(image_path, label, X_train, y_train, image_resolution)
        elif 'test' in image_path:
            resize_and_append(image_path, label, X_test, y_test, image_resolution)
        elif 'val' in image_path:
            resize_and_append(image_path, label, X_val, y_val, image_resolution)
            
    # convert lists to numpy arrays
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    X_val = np.array(X_val)
    y_val = np.array(y_val)
    
    # label encode the target variable (use sparse_categorical_crossentropy as loss function for multiclass)
    y_train, y_test, y_val = label_encode(y_train, y_test, y_val)
    
    return X_train, y_train, X_test, y_test, X_val, y_val

def data_augmentation(X_train, y_train, datagen, augmented_images_per_image):
    """
    Perform data augmentation on the training dataset.
    
    Parameters:
        - X_train (numpy.ndarray): Array of training images.
        - y_train (numpy.ndarray): Array of training labels.
        - datagen (ImageDataGenerator): Keras ImageDataGenerator instance for generating augmented images.
        - augmented_images_per_image (int): Number of augmented images to generate per original image.
        
    Returns:
        - X_train_augmented (numpy.ndarray): Array of augmented training images.
        - y_train_augmented (numpy.ndarray): Array of augmented training labels.
    """
    augmented_images = []
    augmented_labels = []

    # also include original images in the data augmentation
    for i in range(len(X_train)):
        augmented_images.append(X_train[i])
        augmented_labels.append(y_train[i])

    # generate augmented images
    for i in range(len(X_train)):
        x = X_train[i]
        y = y_train[i]
        x = x.reshape((1,) + x.shape) # datagen.flow expects 4D arrays, so we need to reshape the 3D array
        boda = 0
        for batch in datagen.flow(x, batch_size=1): # generate 1 augmented image per iteration
            augmented_images.append(batch[0])
            augmented_labels.append(y)
            boda += 1
            if boda >= augmented_images_per_image:
                break

    X_train_augmented = np.array(augmented_images)
    y_train_augmented = np.array(augmented_labels)
    
    return X_train_augmented, y_train_augmented

def preproc_pipeline(desired_magnification, 
                     image_resolution, 
                     classification_type='binary',
                     use_data_augmentation=False,
                     augmented_images_per_image=5,
                     csv_path = 'image_metadata/updated_image_data.csv',
                     batch_size=32):
    """
    Preprocess image data.

    This function loads image data from the CSV file containing image metadata, filters it based on the desired magnification,
    resizes the images, normalizes pixel values, encodes labels, and optionally performs data augmentation on the training set.
    It returns data generators for training and validation, as well as the test dataset and class weights.

    Parameters:
        - desired_magnification (int): The magnification level to filter the images (40X, 100X, 200X, 400X).
        - image_resolution (tuple): The desired resolution to resize the images (width, height).
        - classification_type (str, optional): The type of classification ('binary' or 'multiclass'). Defaults to 'binary'.
        - use_data_augmentation (bool, optional): Whether to perform data augmentation on the training dataset. Defaults to False.
        - augmented_images_per_image (int, optional): Number of augmented images to generate per original image. Defaults to 5.
        - csv_path (str, optional): Path to the image metadata CSV file. Defaults to 'image_metadata/updated_image_data.csv'.
        - batch_size (int, optional): The batch size for the data generators. Defaults to 32.

    Returns:
        - train_gen (Iterator): Data generator for the training dataset.
        - val_gen (Iterator): Data generator for the validation dataset.
        - test_gen (Iterator): Data generator for the test dataset.
        - class_weights (dict): Dictionary of class weights to handle class imbalance.
    """
    
    if classification_type == 'binary':
        label_column = 'Benign or Malignant'
    else: 
        classification_type = 'multiclass'
        label_column = 'Cancer Type'
    
    X_train, y_train, X_test, y_test, X_val, y_val = load_and_preprocess_data(csv_path, desired_magnification, image_resolution, label_column)
    
    datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    )
    
    if use_data_augmentation == True:
        print(f'Number of training images before data augmentation: {len(X_train)}')
        X_train, y_train = data_augmentation(X_train, y_train, datagen, augmented_images_per_image)
        print(f'Number of training images after data augmentation: {len(X_train)}')
    
    # calculate class weights because our problem is unbalanced
    # np.unique makes this work for both binary (Benign/Malignant) and multiclass classification (Cancer Type)
    class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weights = {i: weight for i, weight in enumerate(class_weights)}

    
    # data augmentation generators
    # shuffles the data so no need to shuffle the data before passing it to the generator
    train_gen = datagen.flow(X_train, y_train, batch_size=batch_size, shuffle=True)
    
    # define a generator for the validation and test data (only rescale)
    datagen_clean_pass = ImageDataGenerator(rescale=1./255)
    
    val_gen = datagen_clean_pass.flow(X_val, y_val, batch_size=batch_size, shuffle=True)
    test_gen = datagen_clean_pass.flow(X_test, y_test, batch_size=batch_size)
    
    return train_gen, val_gen, test_gen, class_weights

print(train_gen.class_indices)

AttributeError: 'NumpyArrayIterator' object has no attribute 'class_indices'